In [1]:
import numpy as np
import pandas as pd

In [2]:
patent_abstract = pd.read_csv('US_patent_abstract_5000_2015_with_title_1_5y.csv')
patent_abstract.shape

(5000, 30)

In [3]:
patent_abstract['quality_rank'].value_counts()

0    2992
1    2008
Name: quality_rank, dtype: int64

In [4]:
2992/5000

0.5984

In [5]:
data_train = patent_abstract

In [6]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using TensorFlow backend.


In [7]:
MAX_SENT_LENGTH = 30
MAX_SENTS = 30
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [8]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [9]:
import nltk
from nltk import tokenize

claims = [] # abstracts is list of list of list to hold each sentences of each abstract (the most complete data)
labels = [] # label is just a list holding our label which is quality_index
texts = []  # texts to hold each complete abstract as list of list (note: abstract not breaking up to sentence level)
for idx in range(data_train.text.shape[0]): # for each row
    text = clean_str(str(data_train.iloc[idx]['text'])) # text is each complete abstract
    texts.append(text) # texts to hold each complete abstract as list of string (note: abstract not breaking up to sentence level)
    sentences = tokenize.sent_tokenize(text) # sentences is list of string holding each complete sentence 
    claims.append(sentences) # abstracts is list of list of string to hold each sentences of each abstract (the most complete data)
    labels.append(data_train.iloc[idx]['quality_rank']) # label is just a list holding our label which is quality_index

In [10]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) # intend to use next line .fit_on_texts to index each word within specific abstract at current iteration/loop, the more frequent word has lower index number, it is a dictionary format, it's like a unique vocabulary index
tokenizer.fit_on_texts(texts)
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [11]:
tokenizer.word_counts

OrderedDict([('invitation', 11),
             ('information', 1296),
             ('push', 23),
             ('method', 3625),
             ('and', 19869),
             ('system', 2717),
             ('an', 7963),
             ('includes', 3102),
             ('after', 218),
             ('receiving', 613),
             ('request', 266),
             ('sent', 61),
             ('by', 3106),
             ('a', 36677),
             ('microblog', 5),
             ('user', 1000),
             ('server', 297),
             ('sending', 58),
             ('to', 14897),
             ('number', 305),
             ('of', 20895),
             ('clients', 16),
             ('corresponding', 447),
             ('invited', 3),
             ('users', 100),
             ('carried', 45),
             ('in', 8248),
             ('the', 46915),
             ('wherein', 796),
             ('are', 2868),
             ('who', 24),
             ('have', 332),
             ('not', 429),
             ('registe

In [12]:
tokenizer.document_count

5000

In [13]:
len(tokenizer.word_index) # 143937 unqiue vocabulary out of the entire abstract dataset

16864

In [14]:
for i, sentences in enumerate(claims):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
            #for word in wordTokens:
                if word in tokenizer.word_index.keys():
                    if (k < MAX_SENT_LENGTH) and (tokenizer.word_index[word] < MAX_NB_WORDS):
                        data[i, j, k] = tokenizer.word_index[word]
                        k = k + 1

In [15]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 16864 unique tokens.


In [16]:
np.unique(np.asarray(labels))

array([0, 1])

In [17]:
labels = to_categorical(np.asarray(labels), num_classes=2) 
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5000, 30, 30)
Shape of label tensor: (5000, 2)


In [18]:
#indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
#data = data[indices]
#labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [19]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set
[2397. 1603.]
[595. 405.]


In [20]:
x_train[2]

array([[9312,   31,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   2,   12,    9, 9312,   26,   12,   32,   93,    2, 1055,   50,
           2,   10,   66,    4,    2,   14,   66,    1, 1055,  576,    2,
        6134,  913,    4,   95,  461, 4739,    1, 6134],
       [   1, 6134,  913,   32,   25, 2660,   11,    1,  668,  181,    3,
           1, 1055,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   1, 6134,  913,   32,   25,  307,   41,    1,   95,  461, 4739,
           4,   39,    3,    1,  461, 4739,   32,   25,  425,    5,    1,
        6134,  913,    0,    0,    0,    0,    0,    0],
       [   1, 1055,   32,  996,    2, 2747,  626,   26,   32, 1127,   20,
           1,   10,   66,    3,    1, 1055,    5,    1,   14,   66,    3,
           1, 1055,    0,    0, 

In [21]:
x_val[10]

array([[ 5832,    33,     9,    83,    27,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [    2,  5832,    33,    22,     2,    27,   523,     5,   232,
            2,    27,   422,   190,   422,    45,   153,    27,     4,
          153,    27,    45,     2,  5041,  7791,     5,   360,  5041,
            3,     1,    27],
       [    7,   398,     5, 15880,  1917,    27,   387,   535,     1,
          407,     3,     1,    91,  1187,     4,  2432,     3,     1,
          175,    24,     1,  5832,    33,   122,    92,     2,  2629,
         7791,     5,   360],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [    0,     0,     0,

In [22]:
x_train.shape

(4000, 30, 30)

In [23]:
np.unique(y_val, return_counts=True)

(array([0., 1.], dtype=float32), array([1000, 1000]))

In [24]:
# Standard python helper libraries.
import os, sys, re, json, time
import itertools, collections
from importlib import reload
from IPython.display import display

# NumPy and SciPy for matrix ops

import scipy.sparse


# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
import glove_helper; reload(glove_helper)

<module 'glove_helper' from '/home/nobu_yamaguchi/final_project/glove_helper.py'>

In [25]:
GLOVE_DIR = "data/glove"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [26]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [27]:

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
print(embedding_matrix.shape)
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

(16865, 100)


In [28]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=False)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

claims_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
claims_encoder = TimeDistributed(sentEncoder)(claims_input)
l_lstm_sent = Bidirectional(LSTM(100))(claims_encoder)
l_lstm_dense = Dense(100, activation='relu')(l_lstm_sent)
l_lstm_dense = Dense(100, activation='relu')(l_lstm_dense)
l_lstm_dense = Dense(100, activation='relu')(l_lstm_dense)
l_lstm_sent_drop = Dropout(rate=0.3)(l_lstm_dense)
preds = Dense(2, activation='sigmoid')(l_lstm_sent_drop) # [Steven] why 5 will throw error and require 6 ? it is weird since we only have 5 number scale
model = Model(claims_input, preds)

In [29]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [30]:
print("model fitting - Hierachical LSTM max_sent = 30")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=50)

model fitting - Hierachical LSTM max_sent = 30
Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 70s 17ms/step - loss: 0.6779 - acc: 0.5931 - val_loss: 0.6745 - val_acc: 0.5950
Epoch 2/10
4000/4000 [==============================] - 65s 16ms/step - loss: 0.6732 - acc: 0.5978 - val_loss: 0.6786 - val_acc: 0.5950
Epoch 3/10
4000/4000 [==============================] - 66s 16ms/step - loss: 0.6723 - acc: 0.5987 - val_loss: 0.6740 - val_acc: 0.5950
Epoch 4/10
4000/4000 [==============================] - 67s 17ms/step - loss: 0.6634 - acc: 0.6104 - val_loss: 0.6705 - val_acc: 0.5825
Epoch 5/10
4000/4000 [==============================] - 67s 17ms/step - loss: 0.6555 - acc: 0.6200 - val_loss: 0.6655 - val_acc: 0.5925
Epoch 6/10
4000/4000 [==============================] - 67s 17ms/step - loss: 0.6515 - acc: 0.6168 - val_loss: 0.6658 - val_acc: 0.5935
Epoch 7/10
4000/4000 [==============================] - 66s 16ms/step - loss: 0.6329 - ac

In [31]:
pred_test = model.predict(x_val)
np.savetxt('LSTM_abst.csv', pred_test)

In [32]:
model.save("model/LSTM_5y_abst")